# Bounding configuration GP example

This notebook is designed to test whether a simple simple comprised of bounding boxes can be generated using [deap](http://deap.readthedocs.io/en/master/api/tools.html), a Python Evolutionary Algorithm Package.

In [1]:
import copy

import numpy as np

from functools import partial

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp


In [2]:
# def progn(*args):
#     for arg in args:
#         arg()

# def prog2(out1, out2): 
#     return partial(progn,out1,out2)

# def prog3(out1, out2, out3):     
#     return partial(progn,out1,out2,out3)

# def if_then_else(condition, out1, out2):
#     out1() if condition() else out2()
def evalf(*args):
    for arg in args:
        arg()



In [4]:
from OCC.gp import gp_Pnt
from airconics.base import AirconicsShape, AirconicsCollection
import OCC
from airconics import AirCONICStools as act
from OCC.BRepPrimAPI import BRepPrimAPI_MakeBox


# Create a simple Box class that inherits from AirconicsShape
#  - this will behave similarly to the Engine, Fuselage class etc.
class Box(AirconicsShape):
    def __init__(self, xmin, ymin, zmin, dx, dy, dz):
        # This implicitly calls build
        super(Box, self).__init__(construct_geomtry=True,
                                             xmin=xmin,
                                             ymin=ymin,
                                             zmin=zmin,
                                             dx=dx,
                                             dy=dy,
                                             dz=dz)
        
    def Build(self):
        Xmin = gp_Pnt(self.xmin, self.ymin, self.zmin)
        Xmax = gp_Pnt(self.xmin + self.dx, self.ymin + self.dy, self.zmin + self.dz)
        self['Box'] = BRepPrimAPI_MakeBox(Xmin, Xmax).Shape()

        

# def progn(*args):
#     """LSIP progn connective"""
#     for arg in args:
#         arg()



# def fit2(out1, out2):
#     return partial(fitN, out1, out2)

# def fit2(out1, out2):
#     return partial(fitN, out1, out2)

        
class BoxLayout(AirconicsCollection):
    def __init__(self, maxcomponents):
        """maxcompoents is the number of components"""
        super(BoxLayout, self).__init__(parts={}, construct_geometry=False)
        self.maxcomponents = maxcomponents
        
        print("hello")
        
        # Start with a simple box
        self.nparts = 0
        self.routine = None
    
    def run(self, routine):
        self._reset()
        while self.nparts < self.maxcomponents:
            routine()

    def _reset(self):
        self._Parts = {}
        self.nparts = 0

    def boxN(self, xmin, ymin, zmin, dx, dy, dz, *args):
        # Fits N new components to this box layout
        box = Box(xmin, ymin, zmin, dx, dy, dz)
        self.nparts += 1
        self['box' + str(len(args))] = box
        for arg in args:
            arg()
    
    def box0(self, xmin, ymin, zmin, dx, dy, dz):
        box = Box(xmin, ymin, zmin, dx, dy, dz)
        self.nparts += 1
        self['box0'] = box
        return partial()
        
    # Now we need to explicitly add the box1, box2, .... boxM functions,
    # So that they can be added to the GP toolbox
    def box1(self, xmin, ymin, zmin, dx, dy, dz, out1):
        box = Box(xmin, ymin, zmin, dx, dy, dz)
        self.nparts += 1
        self['box1'] = box
        return partial(out1)

    def box2(self, xmin, ymin, zmin, dx, dy, dz, out1, out2):
        box = Box(xmin, ymin, zmin, dx, dy, dz)
        self.nparts += 1
        self['box2'] = box
        return partial(out1, out2)
    
    def temp_fitness(self):
        """Until I come with something useful, the fitness evaluation for this
        configuration will be based on the bounding box volume"""
        xmin, ymin, zmin, xmax, ymax, zmax = self.Extents()
        return (xmax-xmin) * (ymax-ymin)*(zmax-zmin)


In [10]:
config = BoxLayout(maxcomponents=20)
import types, itertools
funct = types.FunctionType
nt = types.NoneType

# I think the second argument means that we have up to 57 floats to use as terminals
pset = gp.PrimitiveSetTyped("MAIN", [], nt)


pset.addPrimitive(config.box0, [float, float, float, float, float, float], nt)
pset.addPrimitive(config.box1, [float, float, float, float, float, float, nt], nt)
pset.addPrimitive(config.box2, [float, float, float, float, float, float, nt, nt], nt)

# terminals
for i in range(20):
    label = 'rand' + str(i)
    pset.addEphemeralConstant(label, np.random.rand, float)
# for i in range(100):
#     pset.addTerminal(1.0, float)

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# Attribute generator
toolbox.register("expr_init", gp.genFull, pset=pset, min_=1, max_=2)

# Structure initializers
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr_init)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


# def evalTopology(individual):
#     # Transform the tree expression to functionnal Python code
#     routine = gp.compile(individual, pset)
#     # Run the generated routine
#     global config
#     config.run(routine)
#     return config.temp_fitness,
    
# toolbox.register("evaluate", evalTopology)
# toolbox.register("select", tools.selTournament, tournsize=7)
# toolbox.register("mate", gp.cxOnePoint)
# toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
# toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

# def main():
#     np.random.seed(69)
    
#     pop = toolbox.population(n=100)
#     hof = tools.HallOfFame(1)
#     stats = tools.Statistics(lambda ind: ind.fitness.values)
#     stats.register("avg", np.mean)
#     stats.register("std", np.std)
#     stats.register("min", np.min)
#     stats.register("max", np.max)
    
#     algorithms.eaSimple(pop, toolbox, 0.5, 0.2, 40, stats, halloffame=hof)
    
#     return pop, hof, stats



Skipping geometry construction for BoxLayout
hello


In [11]:
tree = toolbox.individual()

routine = gp.compile(tree, pset)

config.run(routine)

print(tree)

from airconics.Addons.WebServer.TornadoWeb import TornadoWebRenderer

render = TornadoWebRenderer()

config.Display(renderer)

IndexError: The gp.generate function tried to add a terminal of type '<type 'NoneType'>', but there is none available.

In [12]:
routine = gp.compile(tree, pset)

config.run(routine)
from airconics.Addons.WebServer.TornadoWeb import TornadoWebRenderer

render = TornadoWebRenderer()

config.Display(renderer)

NameError: name 'tree' is not defined

In [23]:
def make_box(xmin, ymin, zmin, dx, dy, dz):
    def box0():
        global config
        config.box0(xmin, ymin, zmin, dx, dy, dz)
    return box0

out1 = make_box(1, 1, 1, 1, 1, 1)

p = config.box1(0,0,0,1,1,1, out1)

from airconics.Addons.WebServer.TornadoWeb import TornadoWebRenderer

renderer = TornadoWebRenderer()

print(config)
config.Display(renderer)

renderer

Skipping geometry construction for Box
['box1']


<TornadoWebRenderer at http://PChambers_ngcm:44678/get/14125560e1e6407b8663501d5e9825ae>

In [15]:
type(config['box1'])

__main__.Box

In [16]:
help(gp.PrimitiveSetTyped)

Help on class PrimitiveSetTyped in module deap.gp:

class PrimitiveSetTyped(__builtin__.object)
 |  Class that contains the primitives that can be used to solve a
 |  Strongly Typed GP problem. The set also defined the researched
 |  function return type, and input arguments type and number.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, name, in_types, ret_type, prefix='ARG')
 |  
 |  addADF(self, adfset)
 |      Add an Automatically Defined Function (ADF) to the set.
 |      
 |      :param adfset: PrimitiveSetTyped containing the primitives with which
 |                     the ADF can be built.
 |  
 |  addEphemeralConstant(self, name, ephemeral, ret_type)
 |      Add an ephemeral constant to the set. An ephemeral constant
 |      is a no argument function that returns a random value. The value
 |      of the constant is constant for a Tree, but may differ from one
 |      Tree to another.
 |      
 |      :param name: name used to refers to this ephemeral type.
 |      :pa

In [17]:
help(gp.PrimitiveSet)

Help on class PrimitiveSet in module deap.gp:

class PrimitiveSet(PrimitiveSetTyped)
 |  Class same as :class:`~deap.gp.PrimitiveSetTyped`, except there is no
 |  definition of type.
 |  
 |  Method resolution order:
 |      PrimitiveSet
 |      PrimitiveSetTyped
 |      __builtin__.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, name, arity, prefix='ARG')
 |  
 |  addEphemeralConstant(self, name, ephemeral)
 |      Add an ephemeral constant to the set.
 |  
 |  addPrimitive(self, primitive, arity, name=None)
 |      Add primitive *primitive* with arity *arity* to the set.
 |      If a name *name* is provided, it will replace the attribute __name__
 |      attribute to represent/identify the primitive.
 |  
 |  addTerminal(self, terminal, name=None)
 |      Add a terminal to the set.
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from PrimitiveSetTyped:
 |  
 |  addADF(self, adfset)
 |      Add an Automatically Define

In [18]:
import itertools

r = [itertools.repeat(float, 57)] + [funct]

for entry in r:
    print(entry)

repeat(<type 'float'>, 57)
<type 'function'>


In [19]:
type(None)

NoneType

In [21]:
from airconics.Addons.WebServer.TornadoWeb import TornadoWebRenderer

renderer = TornadoWebRenderer()
# b = Box(0,0,0,1,1,1)
box = BRepPrimAPI_MakeBox(gp_Pnt(0,0,0), gp_Pnt(1,1,1)).Shape()
renderer.DisplayShape(box)
renderer

<TornadoWebRenderer at http://PChambers_ngcm:40148/get/c21b0223fb084f24839ab7b9c2b06a4f>